In [14]:
import wave
import glob
import random

# for data, model, training
import pandas as pd
import numpy as np
import tensorflow as tf
from scipy import signal

import librosa
import librosa.display

# for visuals and statistics
import matplotlib.pyplot as plt
import seaborn as sns

# Set the seed value for experiment reproducibility.
seed = 42
random.seed(42)
tf.random.set_random_seed(seed)
np.random.seed(seed)

In [15]:
def get_and_shuffle_filenames(dir_name):
    filenames = glob.glob(str(data_dir) + "/*")
    random.shuffle(filenames)
    return filenames

data_dir = "./recordings"
filenames = get_and_shuffle_filenames(data_dir)

print(filenames[:5])

['./recordings/2_jackson_13.wav', './recordings/6_george_34.wav', './recordings/7_george_5.wav', './recordings/1_yweweler_21.wav', './recordings/2_george_42.wav']


In [16]:
# https://www.tensorflow.org/tutorials/audio/simple_audio

def decode_audio(file_path):
    # read file to get buffer                                                                                               
    ifile = wave.open(file_path)
    samples = ifile.getnframes()
    audio = ifile.readframes(samples)

    # convert buffer to float32 using NumPy                                                                                 
    audio_as_np_int16 = np.frombuffer(audio, dtype=np.int16)
    audio_as_np_float32 = audio_as_np_int16.astype(np.float32)
    
    # get largest absolute value
    max_val = np.max(
        np.absolute(
            [np.max(audio_as_np_float32), np.min(audio_as_np_float32)]))
    audio_normalized = audio_as_np_float32 / max_val

    return audio_normalized

def get_label(file_path):
    # label is in the filename
    parts = file_path.split("/")
    label = int(parts[2].split("_")[0])

    return label


In [17]:
# to remove outliers 
X_unfiltered = [(file_path, decode_audio(file_path)) for file_path in filenames]
X_lengths = [audio.shape[0] for _, audio in X_unfiltered]

max_length = int(np.mean(X_lengths) + 2 * np.std(X_lengths))
max_length = int(np.ceil(5861 / 256) * 256)
print(np.mean(X_lengths))
print(np.std(X_lengths))
print(max_length)

3499.4746666666665
1180.9471707171701
5888


In [18]:
def spect(signal):
    spectogram = []
    for i in range(23):
        window_fft = np.fft.rfft(signal[i * 256: (i + 1) * 256])[:-1]
        window_fft = list(np.abs(window_fft))
        spectogram.append(window_fft)
    spectogram = np.array(spectogram)
    spectogram = librosa.amplitude_to_db(spectogram, ref=np.max)
    spectogram = spectogram / 20
    return spectogram

In [19]:
# padding function from
# https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5

X_full = [] # padded X values
y_full = []

numbers = [0] * 10

for file_path, audio in X_unfiltered:
    x_val = audio
    y_val = get_label(file_path)
    
    if y_val > 7:
        continue
    
    signal_length = audio.shape[0]
    if signal_length > max_length:
        numbers[y_val] += 1
    else:
        pad_len = max_length - signal_length
        
        x_val = np.pad(
            x_val, (0, pad_len), 
            'constant', constant_values=(0, 0))
        
#         mel_x = mel_spec(x_val)
        s_x = spect(x_val)
        s_x = s_x.flatten()
        
        X_full.append(s_x)
        y_full.append(y_val)

X_full = np.array(X_full)
y_full = np.array(y_full)

print(X_full.shape)

num_samples, sample_w = X_full.shape
print(num_samples)
print(sample_w)

print(y_full[:10])

(2329, 2944)
2329
2944
[2 6 7 1 2 6 6 4 3 2]


In [20]:
# quantities = {"y": list(range(10)), "quantities": numbers}
df = pd.DataFrame.from_dict({"quantities": numbers})
print(df)
print(sum(numbers))

   quantities
0          10
1           8
2           6
3           5
4           3
5           6
6          22
7          11
8           0
9           0
71


In [21]:
mean = np.mean(X_full)
std = np.std(X_full)
X_full = X_full - mean
X_full = X_full / std

In [22]:
# rows = 3
# cols = 3
# n = rows * cols
# fig, axes = plt.subplots(rows, cols, figsize=(12, 14))

# for i, (audio, label) in enumerate(list(zip(X_full, y_full))[:n]):
#     r = i // cols
#     c = i % cols
#     ax = axes[r][c]
    
#     librosa.display.specshow(audio, sr=8000, hop_length=256, x_axis='time', ax=axes[r][c])
# #     librosa.display.specshow(audio, y_axis='mel', fmax=8000, x_axis='time', ax=axes[r][c]);
# #     plt.title('Mel Spectrogram');
# #     plt.colorbar(format='%+2.0f dB');
    
# #     ax.plot(audio)
# #     ax.set_yticks(np.arange(-1,1.5,0.5))
#     ax.set_title(label)

# plt.show()

In [23]:
tenth = int(num_samples * 0.1)
eightyth = tenth * 8

X_train = X_full[:eightyth]
y_train = y_full[:eightyth]

X_val = X_full[eightyth: eightyth + tenth]
y_val = y_full[eightyth: eightyth + tenth]

X_test = X_full[eightyth + tenth:]
y_test = y_full[eightyth + tenth:]

print('Training set size', len(X_train))
print('Validation set size', len(X_val))
print('Test set size', len(X_test))

Training set size 1856
Validation set size 232
Test set size 241


In [24]:
# INPUTS ARE NORMALIZED

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Reshape((128, 23, 1), input_shape=(sample_w,)))
model.add(tf.keras.layers.Conv2D(64, (4, 4), activation='relu', input_shape=(128, 23, 1)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.AveragePooling2D(2,2))
model.add(tf.keras.layers.Conv2D(16, (4, 4), activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.AveragePooling2D(2,2))
 
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dropout(0.15))    
model.add(tf.keras.layers.Dense(70, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='softmax'))

model.build()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 128, 23, 1)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 20, 64)       1088      
_________________________________________________________________
dropout_3 (Dropout)          (None, 125, 20, 64)       0         
_________________________________________________________________
average_pooling2d_2 (Average (None, 62, 10, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 59, 7, 16)         16400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 59, 7, 16)         0         
_________________________________________________________________
average_pooling2d_3 (Average (None, 29, 3, 16)        

In [25]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=30,
                   validation_data=(X_val, y_val))

Train on 1856 samples, validate on 232 samples
Epoch 1/30
1856/1856 [==============================] - 4s 2ms/sample - loss: 1.9413 - acc: 0.2489 - val_loss: 1.6561 - val_acc: 0.3578
Epoch 2/30
1856/1856 [==============================] - 5s 3ms/sample - loss: 1.3281 - acc: 0.5286 - val_loss: 1.0597 - val_acc: 0.6422
Epoch 3/30
1856/1856 [==============================] - 5s 2ms/sample - loss: 0.9074 - acc: 0.6810 - val_loss: 0.7328 - val_acc: 0.7543
Epoch 4/30
1856/1856 [==============================] - 5s 3ms/sample - loss: 0.6422 - acc: 0.7807 - val_loss: 0.6689 - val_acc: 0.7414
Epoch 5/30
1856/1856 [==============================] - 5s 2ms/sample - loss: 0.4902 - acc: 0.8249 - val_loss: 0.5045 - val_acc: 0.8060
Epoch 6/30
1856/1856 [==============================] - 5s 3ms/sample - loss: 0.3956 - acc: 0.8680 - val_loss: 0.3813 - val_acc: 0.8664
Epoch 7/30
1856/1856 [==============================] - 5s 3ms/sample - loss: 0.3150 - acc: 0.8912 - val_loss: 0.3816 - val_acc: 0.8793
E

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=1)

all_labels = list(range(8))

confusion_mtx = tf.math.confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx,
            xticklabels=all_labels,
            yticklabels=all_labels,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

In [ ]:
model.save("v8_small_7.h5")